# CH02 Pythonic Code

(파이썬스럽다  == pythonic)

- 목표
    - 인덱스와 슬라이스 이해하고 인덱싱 가능한 객체를 올바른 방식으로 구현하기
    - 시퀀스와 이터러블 구현하기
    - 컨텍스트 관리자를 만드는 모범 사례 연구
    - 매직 매서드를 사용해 보다 관용적인 코드 구현
    - 파이썬에서 부작용을 유발하는 흔한 실수 피하기

##### index&slice

In [1]:
my_numbers = (4, 5, 3, 9)       #tuple : hashable

print("negative index :", my_numbers[-1], my_numbers[-3])

my_numbers = (1, 1, 2, 3, 5, 8, 13 , 21)
print("slice :", my_numbers[2:5])
print("slice :", my_numbers[:3])
print("slice :", my_numbers[3:])
print("slice :", my_numbers[::-1])

print("slice :", my_numbers[::])

print("slice :", my_numbers[1:7:2])

interval = slice(1, 7, 2)
print("slice use var interval",my_numbers[interval])

interval = slice(None, 3)
print(my_numbers[interval] == my_numbers[:3])

"""Tip:tuple, string, list의 특정요소를 가져오려한다면 for문보다는 이와 같은 방법 추천"""

negative index : 9 5
slice : (2, 3, 5)
slice : (1, 1, 2)
slice : (3, 5, 8, 13, 21)
slice : (21, 13, 8, 5, 3, 2, 1, 1)
slice : (1, 1, 2, 3, 5, 8, 13, 21)
slice : (1, 3, 8)
slice use var interval (1, 3, 8)
True


'Tip:tuple, string, list의 특정요소를 가져오려한다면 for문보다는 이와 같은 방법 추천'

#### 자체 시퀀스 생성
- 범위로 덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여야 한다
- slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외해야 한다


In [2]:
"""None"""
class Item:
    def __init__(self, *values):
        self._values = list(values)
        
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)
        

In [3]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


In [4]:
"""range의 간격을 지정하면 당연하게도 리스트가 아닌 새로운 range를 얻게 된다."""
range(1, 100)[25:50]

range(26, 51)

#### Context manager(컨텍스트 관리자)
- enter__와 __exit 두 개의 매직 메서드로 구성됨
- with 문은 enter 메서드를 호출하고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당됨
- 해당 블록에 대한 마지막 문장이 끝나면 Context 종료되며 exit 메서드 호출
- 블록 내에 예외 또는 오류가 있는 경우에도 exit 메서드가 여전히 호출되며, 예외는 파라미터로 확인가능
    - True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 정지함을 의미하나  
         좋지 않은 습관

In [5]:
filename = "temp.txt"
fd =open(filename)
try:
    process_file(fd)
finally:
    fd.close()


NameError: name 'process_file' is not defined

In [6]:
""" Context Manager using in DB Dump"""

def stop_database():
    print("systemctl stop postgresql.service")
    
def start_database():
    print("systemctl start postgresql.service")
    
class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()
        
def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()
main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


#### Context manager Implement
- contextlib 모듈을 활용한 Context Manager 구현 가능
    - contextlib.contextmanager decorater를 적용하면 해당 함수의 코드를 Context Manger 로 변환함
    - contextlib.ContextDecorater 를 사용하면 with문 없이 완전히 독립적인 실행 o
        - 컨텍스트 관리자 내부에서 사용하고자 하느 객체를 얻을 수 없는 단점
        - e.g.) with offline_back() as bp : 처럼 사용할 수 없음

    - with contextlib.suppress(DataConversionException) 은 로직 자체적으로 처리하고 있음을 에외임을 명시하고 실패하지 않도록 함

In [7]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield               #아무것도 반환하지 않음
    start_database()
    
        
def main():
    with db_handler():
        db_backup()
    
main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [8]:
import contextlib

class db_handler_decorater(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
        
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@db_handler_decorater()
def offline_backup():
    print("pg_dump database")

#### 프로퍼티, 속성과 객체 매서드의 다른 타입들
- 파이썬 객체의 모든 프로퍼티오 함수는 public
- 밑줄로 시작하는 속성은 해당 객체에 대해 private을 의미하며,  
     외부에서 호출하지 않기를 기대하는것

##### 파이썬에서의 밑줄
- 이중 밑줄은 여러번 확장되는 클래스의 메서드를 이름 충돌 없이  
  오버라이드하기 위해 만들어짐
- 이중 밑줄이 아닌 하나의 밑줄을 사용하는 파이썬의 관습을 지킬 것

In [9]:
class Connecter:
    def __init__(self, source):
        self.source = source
        self._timeout = 60  #하나의 밑줄
        
conn = Connecter("postgresql://localhost")

conn._timeout   = 70
print(conn.source)
print(conn.__dict__)

postgresql://localhost
{'source': 'postgresql://localhost', '_timeout': 70}


In [10]:
class Connecter:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60     # 두 개의 밑줄
        
conn = Connecter("postgresql://localhost")

conn.__timeout
#AttributeError: 'Connecter' object has no attribute '__timeout'

AttributeError: 'Connecter' object has no attribute '__timeout'

In [11]:
conn._Connector__timeout =80
print(conn.__dict__)

{'source': 'postgresql://localhost', '_Connecter__timeout': 60, '_Connector__timeout': 80}


In [12]:
print(vars(conn))
print(conn._Connecter__timeout)

{'source': 'postgresql://localhost', '_Connecter__timeout': 60, '_Connector__timeout': 80}
60


##### 프로퍼티
- 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용
- 자바는 접근 메서드(getter/setter) 파이썬은 프로퍼티

In [13]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")


def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None



class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

user = User("shyeon.kang")
user.email = "shyeon.kang@"


ValueError: 유효한 이메일이 아니므로 shyeon.kang@ 값을 사용할 수 없음

In [14]:
user = User("Jang SeokHee")
user.email = "SeokHee@gmail.com"

#### 이터러블 객체
- 반복 가능한지 확인하기 위해 파이썬은 고수준에서 다음 두 가지를 차례로 검사
    - 객체가 __next__ 나 __iter__ 이터레이터 메소드 중 하나를 포함하는지 여부
    - 객체가 시퀀스이고 __len__과 __getitem__을 모두 가졌는지 여부

##### 이터러블 객체 만들기
- 객체를 반복하려고 하면 파이썬은 해당 객체의 iter() 함수를 호출
- for 문은 StopIteration 예외가 발생할 때까지 next() 호출

In [15]:
from datetime import timedelta
from datetime import date

class DateRangeIterable:
    """ 자체 이터레이터 메서드를 가지고 있음 """

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self # 자신이 이터러블임을 나타냄

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


for 문은 StopAsyncIteration 예외가 발생할 때까지 next()를 호출하는 것과 같으

In [16]:
r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))

In [17]:
next(r), next(r), next(r), next(r)

(datetime.date(2019, 1, 1),
 datetime.date(2019, 1, 2),
 datetime.date(2019, 1, 3),
 datetime.date(2019, 1, 4))

In [18]:
next(r)

StopIteration: 

- 한 번 실행하면 끝의 날짜에 도달한 상태이므로 이후에 호출하면 계속 StopIteration 예외가  
    발생하는 문제
- genorater를 사용하면 이러한 문제 해결 가능 (컨테이너 이터러블)

In [19]:
r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [20]:
max(r1)

ValueError: max() arg is an empty sequence

In [32]:
""" Apply Container iterable """

class DateRangeIterable:
    """자체 이터레이터 메서드를 가지고 있음"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days = 1)

In [33]:

r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [34]:
max(r1)

datetime.date(2019, 1, 4)

##### 사퀀스 만들기
- 아터러블은 메모리를 적게 사용하지만 n번째 요소를 얻기 위한 시간복잡도는 O(n)이다.
- 시퀀스로 구현하면 더 많은 메모리가 사용되지만(모든 것을 보관해야 하므로)  
    특정 요소를 가져오기 위한 indexing 시간복잡도는 O(1)로 상수에 가능하다

In [35]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
        
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
    
    def __getitem__(self, day_no):
        return self._range[day_no]
    
    def __len__(self):
        return len(self._range)
    
    
"""
Tip  두 가지 구현 중 어느 것을 사용할 지 경절할 때 메모리와 CPu 사이의 trade-off
계산해보자. 일반저긍로 이터레이션이 더 좋은 선택이지만(제너레이터는 더욱 바람직하지만)
모든 경우의 요건을 염두에 둬야 한다.
"""
    
    

In [36]:
s1 = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))
for day in s1:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [37]:
s1[0]

datetime.date(2019, 1, 1)

In [38]:
s1[3]

datetime.date(2019, 1, 4)

In [39]:
s1[-1]

datetime.date(2019, 1, 4)

#### 컨테이너 객체
- 컨테이너는 __contain__메서드를 구현한 객체임
- 일반적으로 Boolean 값을 반환하며 in 키워드가 발견될 때 호출됨
    - element in container ==> container.__contains__(elemnt)

In [40]:
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height =height
        
    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height
    
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)
    
    def __contains__(self, coord):
        return coord in self.limits
   
    
def mark_coordiante(grid, coord):
    if coord in grid: print(f"{coord} is in limit")
    else: print(f"{coord} is out of the limit")

In [42]:
grid = Grid(4, 5)

coord = (3, 4)
mark_coordiante(grid, coord)

(3, 4) is in limit


In [43]:
grid = Grid(4, 5)

coord = (5, 7)
mark_coordiante(grid, coord)

(5, 7) is out of the limit


#### 객체의 동적인 속성

- 파이썬은 객체를 호출하면 객체 사전에서  찾아서 __getattribute__를 호출
- 객체에 찾고 있는 속성이 없는 경우 객체 이름을 파라미터로 getattr이라는 추가 메서드가 호출됨

In [45]:
class DynamicAttribute:
    def __init__(self, attribute):
        self.attribute = attribute
        
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음.")


"""Tip : __getattr__ 같은 동적인 메서드를 구현할 때는 AttributeError를 
            발생시켜야 한다는 것에 주의하잔"""

In [46]:
dyn = DynamicAttribute("Value")
dyn.attribute

'Value'

In [47]:
dyn.fallback_test

'[fallback resolved] test'

In [48]:
dyn.__dict__["fallback_new"] = "new value"
dyn.fallback_new

'new value'

In [49]:
getattr(dyn, "something", "default")

'default'

In [51]:
getattr(dyn, "fallback_new", "default")

'new value'

In [52]:
dyn.something

AttributeError: DynamicAttribute에는 something 속성이 없음.

#### 호출형(callable) 객체
- __call__을 사용하면 객체를 일반 함수처럼 호출할 수 있음
- 여기에 전달된 모든 파라미터는 call 메서드에 그대로 전달됨

In [53]:
from collections import defaultdict

class CallCount:
    def __init__(self):
        self._counts = defaultdict(int)

    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]

In [61]:
cc = CallCount()


In [62]:
cc(1)

1

In [63]:
cc(2)

1

In [64]:
cc(1)

2

In [65]:
cc(1)

3

In [66]:
cc("something")

1

#### 파이썬에서 유의할 점

##### 변경 가능한 파라미터의 기본 값

- 변경 가능한 객체를 함수의 기본 인자로 사용하면 안됨 

In [55]:
def wrong_user_display(user_metadata: dict = {"name":"John", "age":30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"{name} ({age})"


In [56]:
wrong_user_display()

'John (30)'

In [57]:
 wrong_user_display({"name":"kang", "age":20})

'kang (20)'

In [58]:
wrong_user_display()

KeyError: 'name'

In [59]:
""" 함수 내로 기본값 설정 """
def good_user_display(user = None):
    user_metadata = {"name":"John", "age":30} # 기본값?

    def display(user_metadata = user_metadata):
        name = user_metadata.pop("name")
        age = user_metadata.pop("age")
        return f"{name} ({age})"

    return display(user)

In [60]:

good_user_display()

AttributeError: 'NoneType' object has no attribute 'pop'

####  내장(built-in) 타입 확장

- 리스트, 문자열, 사전과 같은 내장 타입을 확장하는 올바른 방법은  
     collections 모듈을 사용하는것
- 내장 함수에서 override가 동작하지 않음

In [67]:
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}  "

In [68]:
b1 = BadList((0, 1, 2, 3, 4, 5))

In [69]:
b1[0]

'[even] 0  '

In [70]:
b1[1]

'[odd] 1  '

In [72]:
"".join(b1) # join() 내부의 __getitem__ 함수가 호출되지 않음


#[p70s]
#join은 문자열 리스트를 반복하는 함수.
#반복을 해보며 앞서 정의한 __getitem__이 호출되지 않는다.
#이 문제는 사실 (C에 최적화딘) CPython의  세부 구현 사항이며 PyPy와 같은 플랫폼에서는
# 재현되지 않는다.




TypeError: sequence item 0: expected str instance, int found

In [73]:
""" collections 모듈을 사용하여 문제 해결 """

from collections import UserList

class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}  "

In [74]:
bl = GoodList((0, 1, 2, 3, 4, 5))
"".join(bl)


#dict에서 직접 확장하지 말고 대신 collections.UserDict를 사용해야한다.
# 리스트는 collections.UserList를 사용하고 문자열이라면 collection.UserString을 
# 사용해야한다.

'[even] 0  [odd] 1  [even] 2  [odd] 3  [even] 4  [odd] 5  '